##  📘 AI Handbook Assistant (RAG-Based Project)
**Project:** Generative AI & LLMs  
**Student:** SHUBHAM KUMAR

## Project Overview
This project implements a Retrieval-Augmented Generation (RAG) pipeline that answers user questions based on a set of provided documents (PDF/TXT).
Instead of relying only on an LLM, the system retrieves the most relevant document chunks and uses them as grounded context to generate accurate responses.

**Goal:** Build a clean, modular, and fully functional RAG assistant using embeddings, vector search, and context-based answer generation.


## Objective
- Ingest 2–3 documents (PDF or text files)
- Chunk the text into smaller, meaningful sections
- Generate vector embeddings using sentence-transformers
- Store embeddings in a simple NumPy-based vector search system
- Retrieve top-k relevant chunks for any query
- Produce grounded answers using an LLM (OpenAI API)


In [ ]:

import os
import numpy as np
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
from openai import OpenAI

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxxxxxxxxxxxxxxxx"

from openai import OpenAI
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

## Document Loading
Load PDFs from the folder and extract clean text using PyPDF2.



In [ ]:
from PyPDF2 import PdfReader

def load_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        txt = page.extract_text()
        if txt:
            text += txt + "\n"
    return text

# Path to your single PDF
pdf_path = r"C:\Users\shubh\Documents\ACE_ACADEMY\02_AI_Handbook_Assistant_RAG\python.pdf"

all_text = load_pdf(pdf_path)
print(f"Loaded PDF. Total characters: {len(all_text)}")

## Text Chunking
Split the loaded text into smaller blocks for better retrieval.


In [ ]:
def chunk_text(text, chunk_size=400, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(all_text)
print(f"Total chunks created: {len(chunks)}")

## Embedding Generation
Generate semantic embeddings for each chunk using SentenceTransformers.


In [ ]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
vectors = embed_model.encode(chunks)
vectors = np.array(vectors).astype("float32")
print(f"Embeddings ready. Total vectors: {vectors.shape}")

## Retrieval Function
Retrieve top-k most relevant chunks for a user query using NumPy cosine similarity.


In [ ]:
def retrieve(query, chunks, vectors, k=3):
    query_vec = embed_model.encode([query])[0]
    a_norm = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
    b_norm = query_vec / np.linalg.norm(query_vec)
    sims = np.dot(a_norm, b_norm)
    top_idx = np.argsort(sims)[-k:][::-1]
    return [chunks[i] for i in top_idx]

## RAG Answer Function
Combine retrieved chunks with the query and use the OpenAI LLM to generate a grounded answer.


In [ ]:
def rag_answer(query, chunks, vectors, k=3):
    retrieved = retrieve(query, chunks, vectors, k)
    context = "\n\n---\n".join(retrieved)
    
    prompt = f"""
You are a helpful assistant. Answer the user query using ONLY the information below.

Context:
{context}

User Question: {query}

Final Answer:
"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0
    )
    answer = response.choices[0].message.content
    return answer, retrieved

## Test Queries
Check the RAG pipeline by running a few example queries.


In [ ]:
queries = [
    "What is a variable in Python?",
    "Explain git commit.",
    "How do functions work in Python?"
]

for q in queries:
    print(f"\n=== Query: {q} ===")
    answer, refs = rag_answer(q, chunks, vectors)
    print("\nAnswer:\n", answer)
    print("\nReferences used:\n", refs)

## Notes for Submission

### ✅ What Worked Well
- Text chunking and vector search performed reliably, providing accurate context retrieval.  
- SentenceTransformer embeddings captured semantic meaning effectively, making answers grounded and relevant.  
- The modular pipeline (loading → chunking → embeddings → retrieval → generation) was easy to follow and extend.

### ⚠️ Challenges Faced
- PDF extraction occasionally produced messy or misaligned text, requiring minor cleaning.  
- Embedding generation slowed down with very large PDFs.  
- Chunk size and overlap had to be tuned for optimal retrieval accuracy.  
- Setting up the OpenAI API key on Windows required careful attention.
